In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606610 sha256=3d740865cb41cf52475dc90a7aa756440e8a5da21c8df915dcc925cbbe984fca
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [123]:
data=pd.read_csv('/content/drive/MyDrive/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [124]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, p=np.array(items_weights['weight']), replace=False)
    
    return recs.tolist()

Подготовим датафрейм, который будет аргументом функции

In [91]:
#Вычислим вес в структуре выручки для каждого item

items_weights = data_train.groupby('item_id')['sales_value'].unique().reset_index()
items_weights.columns=['item_id', 'weight']

# суммируем общую выручку для каждого item
items_weights['weight'] = items_weights['weight'].apply(lambda x: sum(x))

#найдем log от суммы выручки item  
items_weights['weight'] = np.log(items_weights['weight'])

#заменил все отрицательные значения логарифма на 0
items_weights.loc[items_weights['weight'] < 0,'weight'] = 0

#найдем сумму log-ов всех items
weight_sum = sum(items_weights['weight'])

#найдем вес логарифма выручки каждого item к общей сумме, чтобы сумма весов равнялась 1
items_weights['weight'] = items_weights['weight'] / weight_sum

items_weights.head(4)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,item_id,weight
0,25671,0.000017
1,26081,0.000000
2,26093,0.000003
3,26190,0.000003


Загрузим рекомендации с вебинара

In [95]:
result=pd.read_csv('/content/drive/MyDrive/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1019719, 2332987, 5550112, 9858801, 1072575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[839002, 13158305, 12354593, 8199063, 15597006]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


Рекомендации сохранены как строки, преобразуем данные, чтобы с ними можно было работать

In [96]:
actual = data_test.groupby('user_id')['item_id'].unique()
result = result.merge(actual, on='user_id')

result.drop(['actual', 'own_purchases'], axis=1, inplace=True)
result.rename({'item_id': 'actual'}, axis=1, inplace=True)

In [98]:
for a in result.columns[1:-1]:
    result.loc[:, a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

Добавим в рекомендации с помощью функции weighted_random_recommendation

In [99]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

CPU times: user 3.5 s, sys: 67.7 ms, total: 3.56 s
Wall time: 3.58 s


In [104]:
result.head(2)

,user_id,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,actual,weighted_random_recommendation
0,1,"[1019719, 2332987, 5550112, 9858801, 1072575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[821867, 834484, 856942, 865456, 889248, 90795...","[1042516, 847071, 1025797, 883195, 1098013]"
1,3,"[839002, 13158305, 12354593, 8199063, 15597006]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[835476, 851057, 872021, 878302, 879948, 90963...","[877601, 16809085, 6424123, 1082691, 12262533]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [105]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [106]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], 5), axis=1).mean(),4)}:{name_col}")

0.0008:random_recommendation
0.1552:popular_recommendation
0.1369:itemitem
0.1329:cosine
0.139:tfidf
1.0:actual
0.0013:weighted_random_recommendation


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code

In [125]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head(2)

,item_id,n_sold
0,25671,6
1,26081,1


In [126]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [127]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [128]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


In [129]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Попробуем взять 3 соседа

In [160]:
model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

In [161]:
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [162]:
%%time

result['itemitem_top5000'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])

CPU times: user 63.2 ms, sys: 0 ns, total: 63.2 ms
Wall time: 63.4 ms


In [163]:
result.apply(lambda x: precision_at_k(x['itemitem_top5000'], x['actual'],  5), axis=1).mean()

0.21897649363368973

Посмотрим CosineRecommender на top_5000 товаров и с 3-мя соседями

In [180]:
%%time

model = CosineRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.68 s, sys: 32.8 ms, total: 2.71 s
Wall time: 2.07 s


In [181]:
%%time

result['cosine_top5000'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])

CPU times: user 79.4 ms, sys: 0 ns, total: 79.4 ms
Wall time: 80.5 ms


In [182]:
result.apply(lambda x: precision_at_k(x['cosine_top5000'], x['actual'],  5), axis=1).mean()

0.16385896180215265